# Reading csv files with pandas

In [1]:
import numpy as np

In [2]:
import pandas as pd

## Precipitation data from Japan Meteorological Agency

For details of the data, refer to `data/rain_tokyo_2020.md`.

### Reading values literally

As the first attempt, we are reading `sum24h_na` and `max60m_na` as integer, because they are encoded as integer.

In [3]:
csv = pd.read_csv(
    'data/rain_tokyo_2020.csv',
    encoding = 'shift_jis',
    header = None,
    skiprows = 6,
    names = ['date','sum24h','sum24h_na','D','E','max60m','max60m_na','H','I'],
    index_col = 'date',
    usecols = ['date','sum24h','sum24h_na','max60m','max60m_na'],
    parse_dates = ['date'],
    dayfirst = False,
    dtype = {
        'sum24h': np.float32,
        'sum24h_na': np.int8,
        'max60m': np.float32,
        'max60m_na': np.int8,
    },
    na_filter = False,
)

In [4]:
csv

,sum24h,sum24h_na,max60m,max60m_na
date,,,,
2020-05-01,0.0,1,0.0,1
2020-05-02,0.0,1,0.0,1
2020-05-03,0.0,1,0.0,1
2020-05-04,2.0,0,1.0,0
2020-05-05,0.0,0,0.0,0
...,...,...,...,...
2020-09-26,6.0,0,1.0,0
2020-09-27,0.5,0,0.5,0
2020-09-28,0.0,1,0.0,1


In `read_csv()`, the parameters `index_col` and `parse_dates` ensure that the `date` column (the first column in the file) be parsed as the datetime value and become the index of the resulting dataframe. The type of the index is `DatetmeIndex` as shown below.

In [5]:
csv.index

DatetimeIndex(['2020-05-01', '2020-05-02', '2020-05-03', '2020-05-04',
               '2020-05-05', '2020-05-06', '2020-05-07', '2020-05-08',
               '2020-05-09', '2020-05-10',
               ...
               '2020-09-21', '2020-09-22', '2020-09-23', '2020-09-24',
               '2020-09-25', '2020-09-26', '2020-09-27', '2020-09-28',
               '2020-09-29', '2020-09-30'],
              dtype='datetime64[ns]', name='date', length=153, freq=None)

### Reading flags as `bool`

Note that `1` in the `*_na` columns means the corresponding values are missing data. So we are reading `sum24h_na` and `max60m_na` as `bool`, rather than integer.

In [6]:
csv = pd.read_csv(
    'data/rain_tokyo_2020.csv',
    encoding = 'shift_jis',
    header = None,
    skiprows = 6,
    names = ['date','sum24h','sum24h_na','D','E','max60m','max60m_na','H','I'],
    index_col = 'date',
    usecols = ['date','sum24h','sum24h_na','max60m','max60m_na'],
    parse_dates = ['date'],
    dayfirst = False,
    dtype = {
        'sum24h': np.float32,
        'sum24h_na': bool,
        'max60m': np.float32,
        'max60m_na': bool,
    },
    na_filter = False,
)

In [7]:
csv

,sum24h,sum24h_na,max60m,max60m_na
date,,,,
2020-05-01,0.0,True,0.0,True
2020-05-02,0.0,True,0.0,True
2020-05-03,0.0,True,0.0,True
2020-05-04,2.0,False,1.0,False
2020-05-05,0.0,False,0.0,False
...,...,...,...,...
2020-09-26,6.0,False,1.0,False
2020-09-27,0.5,False,0.5,False
2020-09-28,0.0,True,0.0,True


Applying the `*_na` flags to the corresponding values.

In [8]:
tidy_rain = (
    csv[['sum24h', 'max60m']]
    .mask((csv[['sum24h_na', 'max60m_na']]).to_numpy())
)

In [9]:
tidy_rain

,sum24h,max60m
date,,
2020-05-01,NaN,NaN
2020-05-02,NaN,NaN
2020-05-03,NaN,NaN
2020-05-04,2.0,1.0
2020-05-05,0.0,0.0
...,...,...
2020-09-26,6.0,1.0
2020-09-27,0.5,0.5
2020-09-28,NaN,NaN


### Reorganizing the DataFrame for presentation

We are organizing the `tidy_rain` into a wider form.

In [10]:
(tidy_rain
    .assign(month = lambda s: s.index.strftime('%Y_%m'))
    .assign(day = lambda s: s.index.strftime('d%d'))
    .reset_index(drop = True)
    .melt(
        id_vars = ['month', 'day'],
        value_vars = ['sum24h', 'max60m'],
        var_name = 'element'
    )
    .pivot(
        index=['month','element'],
        columns='day',
        values='value'
    )
    .rename_axis(None, axis=1)
    .rename_axis([None, None], axis=0)
)

d01   d02  d03   d04  d05   d06   d07  d08   d09  d10  ...  \
2020_05 max60m   NaN   NaN  NaN   1.0  0.0   8.5   0.5  NaN   0.0  1.0  ...   
        sum24h   NaN   NaN  NaN   2.0  0.0  19.0   0.5  NaN   0.0  1.0  ...   
2020_06 max60m   1.5   0.5  0.0   0.0  NaN   5.5   0.5  0.0   NaN  0.0  ...   
        sum24h   4.0   0.0  0.0   0.0  NaN   6.5   0.5  0.0   NaN  0.0  ...   
2020_07 max60m  12.5  12.5  3.0   8.0  1.5  11.0   0.5  2.5   5.0  0.5  ...   
        sum24h  21.0   0.5  6.0  25.5  3.5  40.5   0.5  3.0  14.0  0.5  ...   
2020_08 max60m   0.5   NaN  NaN   NaN  0.0   NaN   NaN  NaN   NaN  NaN  ...   
        sum24h   0.5   NaN  NaN   NaN  0.0   NaN   NaN  NaN   NaN  NaN  ...   
2020_09 max60m   1.5   5.0  1.5   0.0  8.5   8.5  13.0  NaN   0.5  0.0  ...   
        sum24h   1.0   7.5  3.5   0.0  8.5  25.5  19.0  NaN   0.5  0.0  ...   

                 d22   d23   d24   d25   d26  d27   d28  d29   d30  d31  
2020_05 max60m   0.5   0.5   NaN   NaN   0.5  0.0   9.0  NaN   NaN  0.0  
        sum24h   1.0   0.0   NaN   NaN   0.5  0.0   9.5  NaN   NaN  0.0  
2020_06 max60m   7.0   1.5   0.0   8.0   0.0  0.0   6.5  NaN   8.5  NaN  
        sum24h  27.5   3.0   0.0  14.5   0.0  0.0  30.0  NaN  15.5  NaN  
2020_07 max60m   0.0  17.5   0.0  10.0   7.0  1.0   1.0  0.0   0.0  1.0  
        sum24h   0.0  31.5   0.0  23.5  18.5  2.0   2.0  0.0   0.0  1.0  
2020_08 max60m   0.0  34.5   0.0   NaN   NaN  0.0   NaN  NaN   NaN  1.0  
        sum24h   0.0  38.0   0.0   NaN   NaN  0.0   NaN  NaN   NaN  1.0  
2020_09 max60m   1.5   2.0   4.0   1.0   1.0  0.5   NaN  0.0   NaN  NaN  
        sum24h   3.5   7.5  10.0   5.5   6.0  0.5   NaN  0.0   NaN  NaN  

[10 rows x 31 columns]